In [1]:
import torch
print(torch.__version__)

import dgl
import numpy as np
import pandas as pd
import os
from dgl.data import DGLDataset
from sklearn.preprocessing import StandardScaler

from __future__ import division
from __future__ import print_function
import time
import argparse
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pickle
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error


import dgl.nn as dglnn
import torch.nn as nn
from utils import load_data, r2_fun
from models import EGAT

import warnings
warnings.filterwarnings('ignore')

1.13.1


In [2]:
class load_data(DGLDataset):
    def __init__(self):
        super().__init__(name='karate_clube')

    def process(self):
        nodes_data = pd.read_csv('./up/node_data.csv')
        edges_data = pd.read_csv('./up/edge_features2.csv')
        node_fe_contin = ['std_dem','avg_slope', 'std_slope', 'medi_aspec', 'avg_plan_c', 
                        'std_plan_c', 'avg_prof_c','std_prof_c', 'avg_maxpre', 'avg_mean a', 'dis2river', 
                        'max GWS','min GWS', 'differ GWS',  'dis2faults']            #18 ndvi
        node_fe_categ = ['land cover','lithology']   #2
        edge_fe_name = ['area_ratio','centre distance', 'aspect angle differ', 'elevation differ']
        
        node_features_contin = nodes_data[node_fe_contin].to_numpy()
        node_features_categ = torch.from_numpy(nodes_data[node_fe_categ].astype('category').to_numpy())
        
        node_labels = nodes_data['max_vel1'].to_numpy()
        edge_features = edges_data[edge_fe_name].to_numpy()
        edges_src = torch.from_numpy(edges_data['src'].to_numpy())
        edges_dst = torch.from_numpy(edges_data['des'].to_numpy()) 
        
        node_labels = np.reshape(node_labels,(-1,1))
        ss_node = StandardScaler()
        ss_edge = StandardScaler()
        ss_y = StandardScaler()

        node_features_nor = ss_node.fit_transform(node_features_contin)
        edge_features_nor = ss_edge.fit_transform(edge_features)
        node_labels_nor = ss_y.fit_transform(node_labels)
        print(type(node_features_nor))
        self.ss_y = ss_y
        self.y_ture = node_labels

        
        node_features_nor  = torch.from_numpy(node_features_nor.astype('float32'))
        node_labels_nor  = torch.from_numpy(node_labels_nor.astype('float32'))
        edge_features_nor  = torch.from_numpy(edge_features_nor.astype('float32'))

        #cat continues and category features
        print(node_features_nor.shape)
        cont_cat_fe = torch.cat((node_features_nor,node_features_categ/10),dim = 1)
        print(cont_cat_fe.shape)
        

        self.graph = dgl.graph((edges_src, edges_dst), num_nodes=nodes_data.shape[0])
        self.graph.ndata['feat'] = cont_cat_fe
        #self.graph.ndata['feat2'] = node_features_categ
        self.graph.ndata['label'] = node_labels_nor
        self.graph.edata['weight'] = edge_features_nor

    def __getitem__(self, i):
        return self.graph

    def __len__(self):
        return 1

dataset = load_data()
graph = dataset[0]
print(graph)

<class 'numpy.ndarray'>
torch.Size([23140, 15])
torch.Size([23140, 17])
Graph(num_nodes=23140, num_edges=139802,
      ndata_schemes={'feat': Scheme(shape=(17,), dtype=torch.float32), 'label': Scheme(shape=(1,), dtype=torch.float32)}
      edata_schemes={'weight': Scheme(shape=(4,), dtype=torch.float32)})


In [3]:
node_features = torch.FloatTensor(np.array(graph.ndata['feat'].detach()))
edge_features = torch.FloatTensor(np.array(graph.edata['weight']))
labels = torch.FloatTensor(np.array(graph.ndata['label']))

print(node_features.shape)
print(edge_features.shape)
print(labels.shape)

#import random
np.random.seed(113)
idx = np.arange(len(labels))   #which is equal to su_id
no_points_su_id = pd.read_csv('./up/no_label_id_ERA5.csv')
no_points_su_id = list(no_points_su_id['su_id'])
idx = np.delete(idx, np.where(np.isin(idx, no_points_su_id)))
print(len(idx))


part1_size = int(0.4 * len(idx))
part2_size = int(0.1 * len(idx))
part3_size = int(0.5 * len(idx))

np.random.shuffle(idx)

idx_train = idx[:part1_size]
idx_val = idx[part1_size:part1_size+part2_size]
idx_test = idx[part1_size+part2_size:part1_size+part2_size+part3_size]

print(idx_train[:10])

torch.Size([23140, 17])
torch.Size([139802, 4])
torch.Size([23140, 1])
22926
[15190 19205 19790 14477  1196   117   913 14799 16235 14408]


In [14]:
# 定义神经网络模型
class RegressionNN(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(RegressionNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)  # 输入层到第一个隐藏层
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)  # 第一个隐藏层到第二个隐藏层
        self.fc0 = nn.Linear(hidden_dim2, output_dim)  # 第二个隐藏层到输出层
        self.relu = nn.ReLU()  # 激活函数
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)

        #self.embedding_1 = nn.Embedding(10,2)  #9 equal to calsses of land cover
        #self.embedding_2 = nn.Embedding(13,2)  #7 equal to calsses of lithology
        
    def forward(self, x1):
        #embedded_1 = self.embedding_1(x1[:,-2].long())
        #embedded_2 = self.embedding_2(x1[:,-1].long())

        #combined = torch.cat((x1[:,:-2], embedded_1.view(embedded_1.size(0), -1), embedded_2.view(embedded_2.size(0), -1)), dim=1)
        
        x = self.relu(self.fc1(x1))  # first layer
        x= self.dropout1(x)
        x = self.relu(self.fc2(x))  # second layer
        x= self.dropout2(x)

        x = self.fc0(x)  
        return x


input_dim = 17  
hidden_dim1 = 128  
hidden_dim2 = 128 
output_dim = 1  
model = RegressionNN(input_dim, hidden_dim1, hidden_dim2, output_dim)


criterion = nn.MSELoss() 
optimizer = optim.Adam(model.parameters(), lr=0.005,weight_decay = 5e-5)  # Adam

print(model)

RegressionNN(
  (fc1): Linear(in_features=17, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc0): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
  (dropout1): Dropout(p=0.5, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
)


In [15]:
#optimizer = optim.SGD(model.parameters(), lr=0.01,momentum = 0.9,weight_decay=0.00005)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.9)
num_epochs = 400
for epoch in range(num_epochs):
    model.train()
    outputs = model(node_features)
    optimizer.zero_grad()
    # 计算损失
    l = criterion(outputs[idx_train], labels[idx_train])
    l.backward()
    optimizer.step()
    scheduler.step()
    #r2_train = r2_fun(outputs, labels[idx_train])
    
    mse = F.mse_loss(outputs[idx_val], labels[idx_val])
    r1 = r2_fun(outputs[idx_val], labels[idx_val])

    # 打印训练误差
    print(f'-Epoch [{epoch+1}/{num_epochs}], train_Loss: {l:.4f},val_Loss:{mse:.4f}')

-Epoch [1/400], train_Loss: 1.0197,val_Loss:0.9861
-Epoch [2/400], train_Loss: 0.9043,val_Loss:0.8723
-Epoch [3/400], train_Loss: 0.8402,val_Loss:0.8201
-Epoch [4/400], train_Loss: 0.8170,val_Loss:0.7840
-Epoch [5/400], train_Loss: 0.7906,val_Loss:0.7477
-Epoch [6/400], train_Loss: 0.7763,val_Loss:0.7247
-Epoch [7/400], train_Loss: 0.7559,val_Loss:0.7056
-Epoch [8/400], train_Loss: 0.7475,val_Loss:0.7107
-Epoch [9/400], train_Loss: 0.7425,val_Loss:0.6815
-Epoch [10/400], train_Loss: 0.7304,val_Loss:0.6730
-Epoch [11/400], train_Loss: 0.7154,val_Loss:0.6703
-Epoch [12/400], train_Loss: 0.7093,val_Loss:0.6784
-Epoch [13/400], train_Loss: 0.7037,val_Loss:0.6583
-Epoch [14/400], train_Loss: 0.6991,val_Loss:0.6500
-Epoch [15/400], train_Loss: 0.6937,val_Loss:0.6497
-Epoch [16/400], train_Loss: 0.6955,val_Loss:0.6376
-Epoch [17/400], train_Loss: 0.6848,val_Loss:0.6358
-Epoch [18/400], train_Loss: 0.6845,val_Loss:0.6368
-Epoch [19/400], train_Loss: 0.6794,val_Loss:0.6450
-Epoch [20/400], trai

In [16]:
#model.load_state_dict(torch.load('./up/MLP_avg_trained.pth'))
model.eval()
output= model(node_features)
loss1 = F.mse_loss(output[idx_test], labels[idx_test])
r2_val = r2_fun(output[idx_test], labels[idx_test])
s_y = dataset.ss_y
y_pre = s_y.inverse_transform(output.detach().numpy())
y_ture = s_y.inverse_transform(labels.detach().numpy())

r2 = r2_score(y_ture[idx_test], y_pre[idx_test])
mae = mean_absolute_error(y_ture[idx_test], y_pre[idx_test])

print("Mean Squared Error:", r2, mae)

Mean Squared Error: 0.5096277394059817 3.4830086


In [20]:
from torch_geometric.nn import BatchNorm
class GNN(nn.Module):
    def __init__(self, in_feats, hidden_size1,hidden_size2):
        super(GNN, self).__init__()
        
        self.fc1 = nn.Linear(in_feats, hidden_size1)
        self.conv1 = dglnn.GraphConv(hidden_size1, hidden_size2,weight = True)
        self.conv2 = dglnn.GraphConv(hidden_size2, hidden_size2, weight = True)
        #self.conv3 = dglnn.GraphConv(hidden_size2, hidden_size2, weight = True)

        self.fc2 = nn.Linear(128, 1)
        
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        self.dropout3 = nn.Dropout(0.5)
        self.dropout0 = nn.Dropout(0.5)
        self.bn1 = BatchNorm(hidden_size1)

        #self.embedding_1 = nn.Embedding(10,2)  #9 equal to calsses of land cover
        #self.embedding_2 = nn.Embedding(13,2)  #7 equal to calsses of lithology

    def reset_param(self):
        for layer in [self.conv1, self.conv2]:
            nn.itnit.kaiming_uniform_(layer.weight, a=0, mode = 'fan_in',nonlinearity = 'relu')

    def forward(self, g, x1):
        #embedded_1 = self.embedding_1(x1[:,-2].long())
        #embedded_2 = self.embedding_2(x1[:,-1].long())

        #combined_fe = torch.cat((x1[:,:-2], embedded_1.view(embedded_1.size(0), -1), embedded_2.view(embedded_2.size(0), -1)), dim=1)

        #x = self.dropout0(combined_fe)
        x1 =  self.fc1(x1)   #concat combined_fe0 and x2
        x1 =  nn.functional.elu(x1)
        #combined_fe = self.dropout0(combined_fe)
        
        
        x = self.conv1(g, x1)
        #x= self.bn1(x)
        x = torch.relu(x)
        x= self.dropout1(x)
        
        x = self.conv2(g, x)
        x = torch.relu(x)
        x= self.dropout2(x)
        '''
        x = self.conv3(g, x)
        x = torch.relu(x)
        x= self.dropout3(x)
        '''
        #x = self.fc1(x)
        #x = torch.relu(x)
        #x= self.dropout0(x)
        x = self.fc2(x)
        return x

GNN = GNN(in_feats=node_features.shape[1], hidden_size1=128,hidden_size2=128)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(GNN.parameters(), lr=0.005,weight_decay = 5e-5)

In [21]:
#optimizer = optim.SGD(GNN.parameters(), lr=0.01,momentum = 0.9,weight_decay=0.00005)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.9)
for epoch in range(300):
    GNN.train()
    predictions = GNN(graph, node_features)
    loss = loss_fn(predictions[idx_train], labels[idx_train])
    r2_train = r2_fun(predictions[idx_train], labels[idx_train])
    print('------------------epoch:', epoch)
    print(loss.item(),r2_train.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()
    mse = F.mse_loss(predictions[idx_val], labels[idx_val])
    r1 = r2_fun(predictions[idx_val], labels[idx_val])
    print('Mean Squared Error:', mse.item(), r1.item())

------------------epoch: 0
0.9762498140335083 -56.56361118604066
Mean Squared Error: 0.9468945264816284 -60.159629024113904
------------------epoch: 1
0.8659457564353943 -7.7453500601544665
Mean Squared Error: 0.8305231332778931 -7.42079604369712
------------------epoch: 2
0.8354129195213318 -2.8647451814332654
Mean Squared Error: 0.8006804585456848 -2.8087916608365204
------------------epoch: 3
0.7974793314933777 -3.0779929225034754
Mean Squared Error: 0.7656152844429016 -3.0893659878186615
------------------epoch: 4
0.7846793532371521 -3.7231436496126573
Mean Squared Error: 0.753498911857605 -3.5450203989575035
------------------epoch: 5
0.7645609378814697 -3.796322054688142
Mean Squared Error: 0.7223557829856873 -3.5584229040333613
------------------epoch: 6
0.7428364753723145 -2.8710158463759554
Mean Squared Error: 0.702528715133667 -2.7225775560104273
------------------epoch: 7
0.7357779741287231 -1.9619801379478017
Mean Squared Error: 0.7031760811805725 -1.7745637624263773
------

In [22]:
from sklearn.metrics import r2_score,mean_squared_error

GNN.eval()
output= GNN(graph,node_features)
loss_func = torch.nn.MSELoss()

s_y = dataset.ss_y
y_pre = s_y.inverse_transform(output.detach().numpy())
y_ture = s_y.inverse_transform(labels.detach().numpy())
print(y_pre.shape)


r2 = r2_score(y_ture[idx_test], y_pre[idx_test])
mse = mean_squared_error(y_ture[idx_test], y_pre[idx_test])
mae = mean_absolute_error(y_ture[idx_test], y_pre[idx_test])

print("Mean Squared Error:", r2, mae)

(23140, 1)
Mean Squared Error: 0.5084453670515263 3.4681451


In [5]:
# Training settings
parser = argparse.ArgumentParser()
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')
parser.add_argument('--fastmode', action='store_true', default=False,
                    help='Validate during training pass.')
parser.add_argument('--seed', type=int, default=42, help='Random seed.')
parser.add_argument('--epochs', type=int, default=200,
                    help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=0.008,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-5,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--dropout', type=float, default=0.5,
                    help='Dropout rate (1 - keep probability).')
parser.add_argument('--num_heads', type=int, default=8,
                    help='Attention heads.')
parser.add_argument('--l_nums', type=int, default=3,
                    help='Number of EGATLayer.')
parser.add_argument('--lamda', type=float, default=0.85,
                    help='The node feature ratio.')
args =parser.parse_known_args()[0]

args.cuda = not args.no_cuda and torch.cuda.is_available()


np.random.seed(args.seed)
torch.manual_seed(args.seed)
# Load data
if args.cuda:
    torch.cuda.manual_seed(args.seed)
    device = torch.device('cuda:0')
# Model and optimizer
model = EGAT(node_feats=node_features.shape[1],
            edge_feats=edge_features.shape[1],
            f_h = 128,
            f_e = 128,
            lamda = args.lamda,
            num_heads = args.num_heads,
            dropout=args.dropout,
            pred_hid = 128, 
            l_num = args.l_nums, 
             )
optimizer = optim.Adam(model.parameters(),lr=args.lr, weight_decay=args.weight_decay)
#optimizer = optim.SGD(model.parameters(), lr=0.01,momentum = 0.9,weight_decay=args.weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.9)
if args.cuda:
    #print("pppppppppppppppppppppppppppp")
    graph = graph.to(device)
    model = model.to(device)
    node_features = node_features.to(device)
    edge_features = edge_features.to(device)
    labels = labels.to(device)
    idx_train = idx_train.to(device)
    idx_val = idx_val.to(device)
    idx_test = idx_test.to(device)


def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(graph, node_features, edge_features)
    loss_func = torch.nn.MSELoss()
    loss_train = loss_func(output[idx_train], labels[idx_train])
    print("loss------------------------------------------------------111")
    r2_train = r2_fun(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()
    scheduler.step()

    if not args.fastmode:
        model.eval()
        output = model(graph, node_features, edge_features)

    #loss_val = F.cross_entropy(output[idx_val], labels[idx_val]).to(device)
    loss_val = loss_func(output[idx_val],labels[idx_val])
    r2_val = r2_fun(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'r2_train: {:.4f}'.format(r2_train.item()),
         'r2_val: {:.4f}'.format(r2_val.item()))


def test():
    model.eval()
    output = model(graph, node_features, edge_features)
    loss_func = torch.nn.MSELoss()
    loss_test = loss_func(output[idx_test], labels[idx_test])
    r2_test = r2_fun(output[idx_test], labels[idx_test])
    
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
         "r2_loss= {:.4f}".format(r2_test))

In [6]:
#Train model
#optimizer = optim.SGD(model.parameters(), lr=0.01,momentum = 0.9,weight_decay=0.00005)
#optimizer = optim.Adam(model.parameters(), lr=0.005,weight_decay=0.0005)
#print("oooooooooooooooooooo")
print(graph.device)
#print(graph.is_cuda)
t_total = time.time()
for epoch in range(260):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

print('----------')
#Testing
#test()

cpu
torch.Size([139802, 128])
torch.Size([139802, 128])
torch.Size([139802, 128])
loss------------------------------------------------------111
torch.Size([139802, 128])
torch.Size([139802, 128])
torch.Size([139802, 128])
Epoch: 0001 loss_train: 0.9490 loss_val: 0.8696 r2_train: -409.1757 r2_val: -0.7606
torch.Size([139802, 128])
torch.Size([139802, 128])
torch.Size([139802, 128])
loss------------------------------------------------------111
torch.Size([139802, 128])
torch.Size([139802, 128])
torch.Size([139802, 128])
Epoch: 0002 loss_train: 0.9147 loss_val: 1.5039 r2_train: -0.8058 r2_val: -2.6583
torch.Size([139802, 128])
torch.Size([139802, 128])
torch.Size([139802, 128])
loss------------------------------------------------------111
torch.Size([139802, 128])
torch.Size([139802, 128])
torch.Size([139802, 128])
Epoch: 0003 loss_train: 1.5906 loss_val: 0.7314 r2_train: -2.7244 r2_val: -3.6507
torch.Size([139802, 128])
torch.Size([139802, 128])
torch.Size([139802, 128])
loss------------

In [7]:
model.eval()

output = model(graph, node_features, edge_features)
loss_func = torch.nn.MSELoss()

s_y = dataset.ss_y
y_pre = s_y.inverse_transform(output.detach().numpy())
y_ture = s_y.inverse_transform(labels.detach().numpy())
print(y_pre.shape)

r2 = r2_score(y_ture[idx_test], y_pre[idx_test])
mse = mean_squared_error(y_ture[idx_test], y_pre[idx_test])
mae = mean_absolute_error(y_ture[idx_test], y_pre[idx_test])
print("Mean Squared Error:", r2, mae)

torch.Size([139802, 128])
torch.Size([139802, 128])
torch.Size([139802, 128])
(23140, 1)
Mean Squared Error: 0.5634200743379739 3.2398155


In [37]:
#IG explanation
baseline = torch.zeros_like(node_features)
baseline[:,-1] = 0
baseline[:,-2] = 0
print(baseline[:5])
print(baseline.shape)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
torch.Size([23140, 17])


In [44]:
from captum.attr import IntegratedGradients
from functools import partial
ig = IntegratedGradients(partial(model, graph, e_in = edge_features))
mask = ig.attribute(node_features.float(),baselines = baseline,internal_batch_size=128, n_steps=100)

torch.Size([139802, 128])
torch.Size([139802, 128])
torch.Size([139802, 128])
torch.Size([139802, 128])
torch.Size([139802, 128])
torch.Size([139802, 128])
torch.Size([139802, 128])
torch.Size([139802, 128])
torch.Size([139802, 128])
torch.Size([139802, 128])
torch.Size([139802, 128])
torch.Size([139802, 128])
torch.Size([139802, 128])
torch.Size([139802, 128])
torch.Size([139802, 128])
torch.Size([139802, 128])
torch.Size([139802, 128])
torch.Size([139802, 128])
